# Analise de dados imputados

In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.interpolate import Akima1DInterpolator
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Carregando e preparando os dados
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'PM10', 'Monóxido de Carbono','Data e Hora'],
                          parse_dates=['Data e Hora'],
                          index_col='Data e Hora',
                          low_memory=False)

df_original.sort_index(inplace=True)

# Selecionando todas as colunas e focando em um período específico
df = df_original.loc['2019-01-01':'2021-01-31']
df = df.apply(pd.to_numeric, errors='coerce')
df = df.asfreq('h')

# Encontrar a sequência mais longa de dados não ausentes para PM2.5
mask = df['PM2.5'].notna()
id_groups = mask.ne(mask.shift()).cumsum()
longest_sequence = df[mask].groupby(id_groups).filter(lambda x: len(x) == mask.groupby(id_groups).size().max())

print(f"Tamanho da sequência mais longa: {len(longest_sequence)}")
print(f"Número de valores não nulos na sequência mais longa: {longest_sequence['PM2.5'].notna().sum()}")

if len(longest_sequence) == 0:
    raise ValueError("A sequência mais longa está vazia. Verifique seus dados.")

# Métodos de imputação
methods = {
    'Original': None,
    'Linear Interpolation': lambda x: x.interpolate(method='linear'),
    # 'Random Forest': IterativeImputer(estimator=RandomForestRegressor(n_estimators=1000), random_state=0, max_iter=100),

}

# Função para avaliar os métodos de imputação
def evaluate_imputation(original, imputed):
    mask = ~np.isnan(original) & ~np.isnan(imputed)
    original = original[mask]
    imputed = imputed[mask]

    if len(original) == 0 or len(imputed) == 0:
        print("Aviso: Não há dados válidos para comparação após a remoção de NaN.")
        return np.nan, np.nan

    mse = mean_squared_error(original, imputed)
    mae = mean_absolute_error(original, imputed)
    return mse, mae

# Criar dados artificiais com valores ausentes na sequência mais longa
test_data = longest_sequence.copy()
np.random.seed(0)
mask = np.random.rand(len(test_data)) < 0.15
test_data.loc[mask, 'PM2.5'] = np.nan

print(f"Porcentagem de valores ausentes nos dados de teste: {test_data['PM2.5'].isna().mean()*100:.2f}%")

# Avaliar cada método
results = {}
imputed_data = {}
for method_name, method in methods.items():
    if method_name == 'Original':
        continue

    try:
        if isinstance(method, SimpleImputer) or isinstance(method, IterativeImputer) or isinstance(method, KNNImputer):
            imputed = pd.DataFrame(method.fit_transform(test_data), index=test_data.index, columns=test_data.columns)
        else:
            imputed = test_data.copy()
            for col in test_data.columns:
                imputed[col] = method(test_data[col])

        mse, mae = evaluate_imputation(longest_sequence['PM2.5'], imputed['PM2.5'])
        results[method_name] = {'MSE': mse, 'MAE': mae}
        imputed_data[method_name] = imputed
    except Exception as e:
        print(f"Erro ao avaliar o método {method_name}: {str(e)}")
        results[method_name] = {'MSE': np.nan, 'MAE': np.nan}

# Ordenar os resultados pelo MSE
results_df = pd.DataFrame(results).T.sort_values('MSE')

print("Resultados ordenados pelo MSE:")
print(results_df)


Epoch 50/500, Loss: 65.4891
Epoch 100/500, Loss: 52.4965
Epoch 150/500, Loss: 45.4017
Epoch 200/500, Loss: 42.5046
Epoch 250/500, Loss: 36.4222
Epoch 300/500, Loss: 32.3127
Epoch 350/500, Loss: 25.3157
Epoch 400/500, Loss: 21.8768
Epoch 450/500, Loss: 17.8645
Epoch 500/500, Loss: 14.7313

Resultados da Imputação:
                      MSE       MAE
Neural Network  10.318298  0.592693

Número de NaNs nos dados imputados: 0


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# Dataset para PyTorch
class TimeSeriesDataset(Dataset):
    def __init__(self, data, lookback=5):
        self.data = data
        self.lookback = lookback

    def __len__(self):
        return len(self.data) - self.lookback

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.lookback]
        y = self.data[idx + self.lookback]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Rede Neural para imputação
class ImputationNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=32):
        super(ImputationNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# Função para treinar a rede neural
def train_nn(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y in train_loader:
            x = x.unsqueeze(-1)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

def impute_with_nn(model, data, lookback=5):
    model.eval()
    imputed = data.copy()

    # Primeira passagem: usar média móvel para valores iniciais
    rolling_mean = pd.Series(data).rolling(window=lookback*2, center=True, min_periods=1).mean()
    imputed[np.isnan(data)] = rolling_mean[np.isnan(data)]

    # Segunda passagem: refinamento com a rede neural
    for _ in range(2):  # Múltiplas passagens para refinar a imputação
        for i in range(lookback, len(data)):
            if np.isnan(data[i]):
                window = imputed[i-lookback:i]
                if not np.any(np.isnan(window)):  # Verificar se a janela está completa
                    x = torch.tensor(window, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
                    with torch.no_grad():
                        prediction = model(x).item()
                    imputed[i] = prediction

    # Terceira passagem: interpolar quaisquer NaNs remanescentes
    imputed = pd.Series(imputed).interpolate(method='linear', limit_direction='both').values

    return imputed

# Função para avaliar os métodos de imputação
def evaluate_imputation(original, imputed):
    mask = ~np.isnan(original) & ~np.isnan(imputed)
    original = original[mask]
    imputed = imputed[mask]

    if len(original) == 0 or len(imputed) == 0:
        print("Aviso: Não há dados válidos para comparação após a remoção de NaN.")
        return np.nan, np.nan

    mse = mean_squared_error(original, imputed)
    mae = mean_absolute_error(original, imputed)
    return mse, mae

# Preparação e treinamento
def prepare_and_train_model(data, lookback=24, hidden_size=64, epochs=500):
    # Remover NaNs para treinamento
    train_data = data[~np.isnan(data)]

    # Criar dataset e dataloader
    dataset = TimeSeriesDataset(train_data, lookback=lookback)
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Inicializar modelo e otimizador
    model = ImputationNN(input_size=1, hidden_size=hidden_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Treinar modelo
    train_nn(model, train_loader, criterion, optimizer, epochs=epochs)

    return model

# Carregando e preparando os dados
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'PM10', 'Monóxido de Carbono','Data e Hora'],
                          parse_dates=['Data e Hora'],
                          index_col='Data e Hora',
                          low_memory=False)

df_original.sort_index(inplace=True)
df = df_original.loc['2019-01-01':'2021-01-31']
df = df.apply(pd.to_numeric, errors='coerce')
df = df.asfreq('h')

# Encontrar a sequência mais longa de dados não ausentes para PM2.5
mask = df['PM2.5'].notna()
id_groups = mask.ne(mask.shift()).cumsum()
longest_sequence = df[mask].groupby(id_groups).filter(lambda x: len(x) == mask.groupby(id_groups).size().max())

# Criar dados artificiais com valores ausentes na sequência mais longa
test_data = longest_sequence.copy()
np.random.seed(0)
mask = np.random.rand(len(test_data)) < 0.15
test_data.loc[mask, 'PM2.5'] = np.nan

# Preparar dados para a rede neural
data_nn = test_data['PM2.5'].values

# Treinar o modelo
model = prepare_and_train_model(data_nn, lookback=48, hidden_size=64, epochs=1000)

# Realizar a imputação
imputed_nn = impute_with_nn(model, data_nn, lookback=48)

# Avaliar resultados
mse_nn, mae_nn = evaluate_imputation(longest_sequence['PM2.5'].values, imputed_nn)

# Criar e exibir DataFrame com resultados
results = {'Neural Network': {'MSE': mse_nn, 'MAE': mae_nn}}
results_df = pd.DataFrame(results).T.sort_values('MSE')
print("\nResultados da Imputação:")
print(results_df)

# Verificar se ainda existem NaNs nos dados imputados
nan_count = np.isnan(imputed_nn).sum()
print(f"\nNúmero de NaNs nos dados imputados: {nan_count}")

# Opcional: Salvar os resultados
results_series = pd.Series(imputed_nn, index=test_data.index)
results_series.to_csv('resultados_imputacao_nn.csv')

Epoch 50/1000, Loss: 90.6340
Epoch 100/1000, Loss: 61.8801
Epoch 150/1000, Loss: 50.0071
Epoch 200/1000, Loss: 44.3313
Epoch 250/1000, Loss: 37.1721
Epoch 300/1000, Loss: 31.8426
Epoch 350/1000, Loss: 25.5996
Epoch 400/1000, Loss: 19.9158
Epoch 450/1000, Loss: 13.9940
Epoch 500/1000, Loss: 11.5305
Epoch 550/1000, Loss: 10.5779
Epoch 600/1000, Loss: 8.3290
Epoch 650/1000, Loss: 10.9780
Epoch 700/1000, Loss: 6.5418
Epoch 750/1000, Loss: 5.1755
Epoch 800/1000, Loss: 3.8154
Epoch 850/1000, Loss: 3.0580
Epoch 900/1000, Loss: 2.8154
Epoch 950/1000, Loss: 3.1698
Epoch 1000/1000, Loss: 2.2004

Resultados da Imputação:
                      MSE       MAE
Neural Network  13.009837  0.816325

Número de NaNs nos dados imputados: 0


In [43]:
import plotly.graph_objects as go

# Criar gráfico comparando os dados originais, ausentes e imputados
fig = go.Figure()

fig.add_trace(go.Scatter(x=longest_sequence.index, y=longest_sequence['PM2.5'], mode='lines', name='Original'))
fig.add_trace(go.Scatter(x=test_data.index, y=data_nn, mode='lines', name='Dados Ausentes'))
fig.add_trace(go.Scatter(x=test_data.index, y=imputed_nn, mode='lines', name='Dados Imputados'))

fig.update_layout(title='Comparação de Dados Originais, Ausentes e Imputados (PM2.5)',
                   xaxis_title='Data e Hora',
                   yaxis_title='PM2.5')

fig.show()
